In [2]:
import psycopg2 as pg2
import psycopg2.extras
import psycopg2.sql

import pandas as pd
import time as time

from algo import run_sql

/home/yuchao/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
conn = None
arch = 'tpch'
qfile_format = 'queries/{arch}/{q}.hypertree'

def use_db(arch, scale):
    global conn
    dbname = 'arch-{arch}-scale-{scale}'.format(arch=arch, scale=scale)
    conn = pg2.connect(dbname=dbname, host='localhost', user='duke', password='duke')

In [4]:
def timeit(f):
    def new_f(*args, **argkv):
        time_start = time.time()
        res = f(*args, **argkv)
        time_finsh = time.time()
        print('Time Elapsed: ', time_finsh - time_start)
        return res
    return new_f

In [5]:
use_db('tpch', '1')

In [6]:
@timeit
def read_sql(sql):
    return pd.read_sql(sql, conn)

In [64]:
@timeit
def read_tables():
    BOTJOIN_node_q2_S_P = pd.read_sql('SELECT * FROM SUPPLIER NATURAL JOIN PART', conn)
    BOTJOIN_node_q2_O_C = pd.read_sql('SELECT * FROM ORDERS NATURAL JOIN CUSTOMER', conn)
    return BOTJOIN_node_q2_S_P, BOTJOIN_node_q2_O_C

In [1]:
BOTJOIN_node_q2_S_P, BOTJOIN_node_q2_O_C = read_tables()

NameError: name 'read_tables' is not defined

In [53]:
BOTJOIN_node_q2_S_P.columns

Index(['partkey', 'suppkey', 'nationkey', 'c_2_3'], dtype='object')

In [54]:
@timeit
def table_join():
    return BOTJOIN_node_q2_S_P.join(BOTJOIN_node_q2_O_C.set_index('nationkey'), on='nationkey')

In [55]:
joined = table_join()

Time Elapsed:  0.13256072998046875


In [ ]:
read_sql('SELECT * FROM BOTJOIN_node_q2_S_P NATURAL JOIN BOTJOIN_node_q2_O_C')

In [ ]:
read_sql('SELECT * FROM SUPPLIER NATURAL JOIN PART')